In [5]:
import pygsheets
import difflib
import re

In [6]:
def client_auth(use_pyg=True):
    # authorization
    # using pygsheets
    if use_pyg:
        gc = pygsheets.authorize(service_file="client_secret.json")
        return gc
    else:
        scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
        client = gspread.authorize(creds)
        return client

In [7]:
def fix_names(name):
    return name.split(' ')

def fix_depts(dept):
    return dept

In [8]:
gc=client_auth()

In [11]:
senate_sh = gc.open('AC Senate Data 2023')
senate_wk = senate_sh[0]
senate_df = senate_wk.get_as_df(header=True)
senate_df

,Class,instructors,Department
0,17AC,VèVè Clark,AFRICAN AMERICAN STUDIES
1,C20AC,Saru Jayaraman and Michael Cohen,AFRICAN AMERICAN STUDIES
2,25AC,William Banks,AFRICAN AMERICAN STUDIES
3,27AC,Percy Hintzen,AFRICAN AMERICAN STUDIES
4,27AC,Tommy Lott,AFRICAN AMERICAN STUDIES
...,...,...,...
1168,20AC,Dawne Moon,UNDERGRADUATE AND INTERDISCIPLINARY STUDIES
1169,56AC,Alex Saragoza,UNDERGRADUATE AND INTERDISCIPLINARY STUDIES
1170,C135,Lewis Watts,UNDERGRADUATE AND INTERDISCIPLINARY STUDIES
1171,C135,Susannah Hays,UNDERGRADUATE AND INTERDISCIPLINARY STUDIES


In [16]:
semester_sh = gc.open("AC Classes List")
sheet_names = [s.title for s in semester_sh.worksheets()]
print(sheet_names)
semester_wk = semester_sh.worksheet_by_title("2023 Fall")
lastest_classes = semester_wk.get_as_df(header=True)
lastest_classes

['Copy of Spring 2020 (as of 11/13/2019)', "Approved['Spring 2020']", "Not Approved['Spring 2020']", '2020 Summer Session', "Not Approved['2020 Summer Session']", "Copy of Not Approved['2020 Summer Session']", "Approved['2020 Summer Session']", '2020 Fall', '2017 Spring', '2017 Summer-6W1', '2017 Fall', '2018 Spring', '2018 Summer Session', '2018 Fall', 'Spring 2019', '2019 Summer Session', '2019 Fall', 'Spring 2020', '2021 Summer Session', '2021 Fall', 'Spring 2021', 'Spring 2022', '2022 Summer Session', '2022 Fall', 'Spring 2023', '2023 Summer Session_ALL', '2023 Fall', '2023 Summer Session']


,Department,Department_FULL,Instruction Mode,Classes Links,Course Number,Instructor 1,Instructor 2,Total Open Seats,Enrolled,Waitlisted,Capacity,Waitlist Max,Open Reserved Seats,Also offered as
0,AFRICAM,AFRICAN AMERICAN STUDIES(OPENS IN A NEW TAB),In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,27AC,Michael M Cohen,NaN,127,73,0,200,30,None Reserved,NaN
1,AFRICAM,AFRICAN AMERICAN STUDIES(OPENS IN A NEW TAB),In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,C133A,Travis Bristol,NaN,0,23,2,23,5,None Reserved,EDUC C181
2,AMERSTD,UNDERGRADUATE INTERDISCIPLINARY STUDIES(OPENS ...,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,101AC,Sarah Erina Gold McBride,NaN,21,29,20,50,20,"21 reserved for American Studies Majors,",NaN
3,AMERSTD,UNDERGRADUATE INTERDISCIPLINARY STUDIES(OPENS ...,In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,102AC,Sarah Erina Gold McBride,NaN,20,30,20,50,20,"21 reserved for American Studies Majors,",NaN
4,ANTHRO,ANTHROPOLOGY(OPENS IN A NEW TAB),Online,https://classes.berkeley.edu//content/2023-fal...,2AC,Christopher Lowman,NaN,17,357,43,374,44,17 reserved for Students with Enrollment Permi...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,XGWS,UC BERKELEY EXTENSION(OPENS IN A NEW TAB),Online,https://classes.berkeley.edu//content/2023-fal...,50AC,NA,NaN,48,0,0,48,0,None Reserved,NaN
63,XHISTOR,UC BERKELEY EXTENSION(OPENS IN A NEW TAB),In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,7B,Samantha H Gervase,NaN,48,0,0,48,8,None Reserved,NaN
64,XHISTOR,UC BERKELEY EXTENSION(OPENS IN A NEW TAB),In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,7B,Samantha H Gervase,NaN,24,0,0,24,4,None Reserved,NaN
65,XMUSIC,UC BERKELEY EXTENSION(OPENS IN A NEW TAB),In-Person Instruction,https://classes.berkeley.edu//content/2023-fal...,26AC,Francesca Rivera,NaN,48,0,0,48,8,None Reserved,NaN


In [ ]:
# Now check if the names are in the Google Spreadsheet
for name, course in names:
    closest_match = difflib.get_close_matches(name, spreadsheet_values, n=1, cutoff=0.6)
    if closest_match:
        print(f'{name} who teaches {course} is probably in the spreadsheet as {closest_match[0]}.')
    else:
        print(f'{name} who teaches {course} is not in the spreadsheet.')